<h2>Lambeth DataNet (LDN)</h2>

London perspective

In [1]:
import csv
from datetime import datetime

In [2]:
import configparser
config = configparser.ConfigParser();
config.read('config.properties');

In [3]:
# SEC distributions
ldn_sec_low_proportion_100, ldn_sec_mid_proportion_100, ldn_sec_high_proportion_100 = 0, 0, 0;
# Ethnicity distributions
ldn_ethnicity_white_british_proportion_100, ldn_ethnicity_minority_proportion_100 = 0, 0;
# SEC + Asthma distributions
ldn_sec_low_asthma_proportion_100, ldn_sec_mid_asthma_proportion_100, ldn_sec_high_asthma_proportion_100, ldn_sec_low_non_asthma_proportion_100, ldn_sec_mid_non_asthma_proportion_100, ldn_sec_high_non_asthma_proportion_100 = 0, 0, 0, 0, 0, 0;
# Ethnicity + Asthma distributions
ldn_ethnicity_white_asthma_proportion_100, ldn_ethnicity_minority_asthma_proportion_100, ldn_ethnicity_white_non_asthma_proportion_100, ldn_ethnicity_minority_non_asthma_proportion_100, = 0, 0, 0, 0;


In [4]:
# load patient data
individuals = {};
class Individual:
    def __init__(self, id, age, sex, lsoa, in_lambeth, ethnicity_a, ethnicity_b):
        self.__id = id;
        self.__age = age;
        self.__sex = sex;
        self.__lsoa = lsoa;
        self.__in_lambeth = in_lambeth;
        self.__ethnicity_a = ethnicity_a;
        self.__ethnicity_b = ethnicity_b;

    def get_id(self): return self.__id;
    def get_age(self): return self.__age;
    def get_sex(self): return self.__sex;
    def get_lsoa(self): return self.__lsoa;
    def get_in_lambeth(self): return self.__in_lambeth;
    def get_ethnicity_a(self): return self.__ethnicity_a;
    def get_ethnicity_b(self): return self.__ethnicity_b;


with open(config.get('data', 'ldn.path_cohort')) as f:
    reader = csv.DictReader(f)
    for row in reader:
        # ~MDC 14/03 there's probably already a package for this
        individuals[row['patient_id']] = Individual(row['patient_id'], int(row['ageatrefdate']), ('M' if int(row['sex']) == 1 else 'F'), row['lsoa2011'], int(row['in_lambeth']), int(row['eth_5']) if row['eth_5'] else -1, int(row['eth_18']) if row['eth_18'] else -1);

# print number of individuals (for interest)
print(len(individuals));

66557


<h3>SEC</h3>

LSOA

In [5]:
# print number of lsoas (just for interest)
with open(config.get('data', 'ldn.path_iod')) as f:
    reader = csv.DictReader(f);
    print(len(list(reader)));

1026


In [6]:
# match each lsoa to an sec group (low, mid or high) based upon 1-10 ranking on employment iod
lsoa_to_sec = {};
with open(config.get('data', 'ldn.path_iod')) as f:
    reader = csv.DictReader(f)
    for row in reader:
        if(not row['iod_employ_decile']): continue;
        iod_employ_decile = int(row['iod_employ_decile']);
        if(iod_employ_decile>=1 and iod_employ_decile<=3):
            lsoa_to_sec[row['lsoa2011']] = 'low';
        elif(iod_employ_decile>=4 and iod_employ_decile<=7):
            lsoa_to_sec[row['lsoa2011']] = 'mid';
        elif(iod_employ_decile>=8 and iod_employ_decile<=10):
            lsoa_to_sec[row['lsoa2011']] = 'high';


In [7]:
# count number of lsoas in each group (just for interest)
sec_grouped = {};
for entry in lsoa_to_sec.items(): 
    sec_grouped.setdefault(entry[1], 0);
    sec_grouped[entry[1]] = sec_grouped[entry[1]] + 1;
print(sec_grouped);

{'mid': 508, 'low': 288, 'high': 198}


In [8]:
ldn_individual_to_lsoa_sec = {};
ldn_lsoa_low, ldn_lsoa_mid, ldn_lsoa_high = 0, 0, 0;
# match individuals to their lsoas and then to each sec
for individual in individuals.values():
    if(not individual.get_lsoa()): continue;
    if(individual.get_lsoa() not in lsoa_to_sec.keys() and individual.get_in_lambeth() == 0): 
        continue;
    elif(individual.get_lsoa() not in lsoa_to_sec.keys() and individual.get_in_lambeth() == 1):
        print('No sec record for lsoa: ' + individual.get_lsoa());
        continue;
    else:
        if(lsoa_to_sec[individual.get_lsoa()] == 'low'): 
            ldn_lsoa_low += 1;
            ldn_individual_to_lsoa_sec[individual.get_id()] = 'low';
        elif(lsoa_to_sec[individual.get_lsoa()] == 'mid'): 
            ldn_lsoa_mid += 1;
            ldn_individual_to_lsoa_sec[individual.get_id()] = 'mid';
        elif(lsoa_to_sec[individual.get_lsoa()] == 'high'): 
            ldn_lsoa_high += 1;
            ldn_individual_to_lsoa_sec[individual.get_id()] = 'high';

In [9]:
print(str(ldn_lsoa_low) + ' ' + str(ldn_lsoa_mid) + ' ' + str(ldn_lsoa_high));

21917 35114 9068


In [10]:
ldn_lsoa_total = ldn_lsoa_low + ldn_lsoa_mid + ldn_lsoa_high;

In [11]:
ldn_lsoa_low_proportion = ldn_lsoa_low / ldn_lsoa_total;
print('LDN LSOA low %: ' + str(ldn_lsoa_low_proportion));

LDN LSOA low %: 0.33157838999077144


In [12]:
ldn_lsoa_mid_proportion = ldn_lsoa_mid / ldn_lsoa_total;
print('LDN LSOA mid %: ' + str(ldn_lsoa_mid_proportion));

LDN LSOA mid %: 0.5312334528510265


In [13]:
ldn_lsoa_high_proportion = ldn_lsoa_high / ldn_lsoa_total;
print('LDN LSOA high %: ' + str(ldn_lsoa_high_proportion));

LDN LSOA high %: 0.1371881571582021


In [14]:
print(ldn_lsoa_low_proportion + ldn_lsoa_mid_proportion + ldn_lsoa_high_proportion);

1.0


BMI

In [15]:
# As the dataset contains multiple readings for the same individuals, match to one reading (the latest)
patient_to_bmi = {};
with open(config.get('data', 'ldn.path_value_clusters'), encoding='utf-8') as f:
    reader = csv.DictReader(f);
    reader_by_date = sorted(reader, key=lambda row: datetime.strptime(row['effectivedate'], '%d/%m/%Y'));
    reader_by_date.reverse();
    for row in reader_by_date:
        if(not row['patient_id'] in patient_to_bmi.keys() and row['numericvalue']):
            patient_to_bmi[row['patient_id']] = float(row['numericvalue']);
print(len(patient_to_bmi));

15160


In [16]:
# overweight classification in children is on a sliding scale, so load this data:
age_bmi_overweight_boys = {};
with open('resources/bmi/boys/age-bmi-overweight.csv') as f:
    reader = csv.DictReader(f);
    age_bmi_overweight_boys = dict(next(reader));
    age_bmi_overweight_boys = dict([int(header), float(cell)] for header, cell in age_bmi_overweight_boys.items());
print(age_bmi_overweight_boys);

age_bmi_overweight_girls = {};
with open('resources/bmi/girls/age-bmi-overweight.csv') as f:
    reader = csv.DictReader(f);
    age_bmi_overweight_girls = dict(next(reader));
    age_bmi_overweight_girls = dict([int(header), float(cell)] for header, cell in age_bmi_overweight_girls.items());
print(age_bmi_overweight_girls);

{2: 18.0, 3: 17.0, 4: 17.0, 5: 17.0, 6: 17.0, 7: 18.0, 8: 18.0, 9: 19.0, 10: 19.0, 11: 20.0, 12: 21.0, 13: 21.0, 14: 22.0, 15: 23.0, 16: 24.0, 17: 24.0, 18: 25.0}
{2: 18.0, 3: 17.0, 4: 17.0, 5: 18.0, 6: 18.0, 7: 18.0, 8: 19.0, 9: 20.0, 10: 20.0, 11: 21.0, 12: 22.0, 13: 23.0, 14: 24.0, 15: 24.0, 16: 25.0, 17: 25.0, 18: 26.0}


In [17]:
# overweight classification in children is on a sliding scale, so load this data:
age_bmi_underweight_boys = {};
with open('resources/bmi/boys/age-bmi-underweight.csv') as f:
    reader = csv.DictReader(f);
    age_bmi_underweight_boys = dict(next(reader));
    age_bmi_underweight_boys = dict([int(header), float(cell)] for header, cell in age_bmi_underweight_boys.items());
print(age_bmi_underweight_boys);

age_bmi_underweight_girls = {};
with open('resources/bmi/girls/age-bmi-underweight.csv') as f:
    reader = csv.DictReader(f);
    age_bmi_underweight_girls = dict(next(reader));
    age_bmi_underweight_girls = dict([int(header), float(cell)] for header, cell in age_bmi_underweight_girls.items());
print(age_bmi_underweight_girls);

{2: 13.0, 3: 13.0, 4: 13.0, 5: 13.0, 6: 13.0, 7: 13.0, 8: 13.0, 9: 13.0, 10: 13.0, 11: 14.0, 12: 14.0, 13: 15.0, 14: 15.0, 15: 16.0, 16: 16.0, 17: 17.0, 18: 17.0}
{2: 13.0, 3: 13.0, 4: 13.0, 5: 13.0, 6: 13.0, 7: 13.0, 8: 13.0, 9: 13.0, 10: 13.0, 11: 14.0, 12: 14.0, 13: 15.0, 14: 15.0, 15: 16.0, 16: 16.0, 17: 16.0, 18: 17.0}


In [18]:
# number of individuals overweight per age (mostly for interest, but also used in final calculation)
ldn_individual_to_bmi_sec = {}; 
def is_overweight(patient_and_bmi):
    return patient_and_bmi[1]>=(age_bmi_overweight_boys[individual.get_age()] if individual.get_sex() == 'M' else age_bmi_overweight_girls[individual.get_age()]);

age_to_overweight = {};
for patient_and_bmi in patient_to_bmi.items():
    individual = individuals[patient_and_bmi[0]];
    if(individual.get_age() < 2): continue;
    age_to_overweight.setdefault(individual.get_age(), 0);
    if(is_overweight(patient_and_bmi)):
        age_to_overweight[individual.get_age()] = age_to_overweight[individual.get_age()] + 1;
        ldn_individual_to_bmi_sec[individual.get_id()] = 'low';
print(dict(sorted(age_to_overweight.items())));

{2: 38, 3: 98, 4: 108, 5: 145, 6: 153, 7: 166, 8: 157, 9: 136, 10: 170, 11: 177, 12: 198, 13: 228, 14: 248, 15: 267, 16: 276, 17: 345, 18: 317}


In [19]:
# number of individuals of a healthy weight per age (mostly for interest, but also used in final calculation)
age_to_healthy_weight = {};
for patient_and_bmi in patient_to_bmi.items():
    individual = individuals[patient_and_bmi[0]];
    if(individual.get_age() < 2): continue;
    age_to_healthy_weight.setdefault(individual.get_age(), 0);
    if((patient_and_bmi[1]<(age_bmi_overweight_boys[individual.get_age()] if individual.get_sex() == 'M' else age_bmi_overweight_girls[individual.get_age()])) and (patient_and_bmi[1]>(age_bmi_underweight_boys[individual.get_age()] if individual.get_sex() == 'M' else age_bmi_underweight_girls[individual.get_age()]))):
        age_to_healthy_weight[individual.get_age()] = age_to_healthy_weight[individual.get_age()] + 1;
        ldn_individual_to_bmi_sec[individual.get_id()] = 'mid or high';
print(dict(sorted(age_to_healthy_weight.items())));

{2: 154, 3: 160, 4: 224, 5: 324, 6: 404, 7: 463, 8: 584, 9: 649, 10: 723, 11: 650, 12: 750, 13: 666, 14: 807, 15: 704, 16: 847, 17: 783, 18: 858}


In [20]:
# consider overweight individuals to have a low sec; record others
ldn_bmi_low, ldn_bmi_mid, ldn_bmi_high = 0, 0 ,0;
ldn_bmi_low = sum(age_to_overweight.values());
ldn_bmi_mid_high = sum(age_to_healthy_weight.values());
print(ldn_bmi_low);
print(ldn_bmi_mid_high);

3227
9750


In [21]:
# we also have explicitly coded overweight individuals. add these if they aren't already flagged
additional_obese_individuals = [];
with open(config.get('data', 'ldn.path_clusters'), encoding='utf-8') as f:
    reader = csv.DictReader(f);
    for row in reader:
        if(row['patient_id'] not in additional_obese_individuals and row['cluster']=='OBESITY' and row['patient_id'] in patient_to_bmi.keys() and not is_overweight((row['patient_id'], patient_to_bmi[row['patient_id']]))):
            ldn_bmi_low = ldn_bmi_low + 1;
            additional_obese_individuals.append(row['patient_id']);
print(len(additional_obese_individuals));
print(ldn_bmi_low);

211
3438


In [22]:
# we can't necessarily make a link between being of a healthy weight and a mid/high SEC, so distribute evenly:
ldn_bmi_mid = ldn_bmi_mid_high / 2;
ldn_bmi_high = ldn_bmi_mid_high / 2;
print(ldn_bmi_mid);
print(ldn_bmi_high);

4875.0
4875.0


In [23]:
ldn_bmi_total = ldn_bmi_low + ldn_bmi_mid + ldn_bmi_high;

In [24]:
ldn_bmi_low_proportion = ldn_bmi_low / ldn_bmi_total;
print('LDN BMI low %: ' + str(ldn_bmi_low_proportion));

LDN BMI low %: 0.2606915377616015


In [25]:
ldn_bmi_mid_proportion = ldn_bmi_mid / ldn_bmi_total;
print('LDN BMI mid %: ' + str(ldn_bmi_mid_proportion));

LDN BMI mid %: 0.36965423111919926


In [26]:
ldn_bmi_high_proportion = ldn_bmi_high / ldn_bmi_total;
print('LDN BMI high %: ' + str(ldn_bmi_high_proportion));

LDN BMI high %: 0.36965423111919926


In [27]:
print(ldn_bmi_low_proportion + ldn_bmi_mid_proportion + ldn_bmi_high_proportion)

1.0


COVID

In [28]:
ldn_covid_low, ldn_covid_mid, ldn_covid_high = 0, 0, 0;
patient_to_covid = {};
ldn_covid_mid_high = 0;
ldn_individual_to_covid_sec = {};
with open(config.get('data', 'ldn.path_covid'), encoding='utf-8') as f:
    reader = csv.DictReader(f);
    reader_by_date = sorted(reader, key=lambda row: datetime.strptime(row['effectivedate'], '%Y-%m-%d'));
    reader_by_date.reverse();
    for row in reader_by_date:
        if('CONF' in row['cluster'] and not row['patient_id'] in patient_to_covid.keys()):
            ldn_covid_low = ldn_covid_low + 1;
            patient_to_covid[row['patient_id']] = True;
            ldn_individual_to_covid_sec[row['patient_id']] = 'low';
        elif('NEGATIVE' in row['cluster'] and not row['patient_id'] in patient_to_covid.keys()):
            ldn_covid_mid_high = ldn_covid_mid_high + 1;
            patient_to_covid[row['patient_id']] = False;
            ldn_individual_to_covid_sec[row['patient_id']] = 'mid or high';
print(ldn_covid_low);
print(ldn_covid_mid_high);

8972
9192


In [29]:
ldn_covid_mid = ldn_covid_mid_high / 2;
ldn_covid_high = ldn_covid_mid_high / 2;
print(ldn_covid_mid);

4596.0


In [30]:
ldn_covid_total = ldn_covid_low + ldn_covid_mid + ldn_covid_high;

In [31]:
ldn_covid_low_proportion = ldn_covid_low / ldn_covid_total;
print('LDN COVID low %: ' + str(ldn_covid_low_proportion));

LDN COVID low %: 0.4939440651838802


In [32]:
ldn_covid_mid_proportion = ldn_covid_mid / ldn_covid_total;
print('LDN COVID mid %: ' + str(ldn_covid_mid_proportion));

LDN COVID mid %: 0.2530279674080599


In [33]:
ldn_covid_high_proportion = ldn_covid_high / ldn_covid_total;
print('LDN COVID high %: ' + str(ldn_covid_high_proportion));

LDN COVID high %: 0.2530279674080599


In [34]:
print(ldn_covid_low_proportion + ldn_covid_mid_proportion + ldn_covid_high_proportion)

0.9999999999999999


Aggregate

In [35]:
# take average of three SEC measures from LDN
ldn_sec_low_proportion = (ldn_lsoa_low_proportion + ldn_bmi_low_proportion + ldn_covid_low_proportion) / 3;
print(ldn_sec_low_proportion);

0.36207133097875105


In [36]:
ldn_sec_mid_proportion = (ldn_lsoa_mid_proportion + ldn_bmi_mid_proportion + ldn_covid_mid_proportion) / 3;
print(ldn_sec_mid_proportion);

0.3846385504594285


In [37]:
ldn_sec_high_proportion = (ldn_lsoa_high_proportion + ldn_bmi_high_proportion + ldn_covid_high_proportion) / 3;
print(ldn_sec_high_proportion);

0.25329011856182043


In [38]:
# double check == 1
print(ldn_sec_low_proportion + ldn_sec_mid_proportion + ldn_sec_high_proportion);

1.0


Distributions

In [39]:
# aggregate for /100
ldn_sec_low_proportion_100 = max(1, int(round(ldn_sec_low_proportion, 2) * 100));
print("LDN low SEC: " + str(ldn_sec_low_proportion_100));
ldn_sec_mid_proportion_100 = max(1, int(round(ldn_sec_mid_proportion, 2) * 100));
print("LDN mid SEC: " + str(ldn_sec_mid_proportion_100));
ldn_sec_high_proportion_100 = max(1, int(round(ldn_sec_high_proportion, 2) * 100));
print("LDN high SEC: " + str(ldn_sec_high_proportion_100));

LDN low SEC: 36
LDN mid SEC: 38
LDN high SEC: 25


In [40]:
print(ldn_sec_low_proportion_100 + ldn_sec_mid_proportion_100 + ldn_sec_high_proportion_100)

99


<h3>Ethnicity</h3>

In [41]:
# combine two ethnicity terms to identify two groups
ldn_ethnicity_white_british, ldn_ethnicity_minority = 0, 0;
ldn_individual_to_ethnicity = {};

for individual in individuals.values():
    if(not individual.get_ethnicity_a() or not individual.get_ethnicity_b()): continue;
    if(individual.get_ethnicity_a()==1 and individual.get_ethnicity_b()==1):
        ldn_ethnicity_white_british = ldn_ethnicity_white_british + 1;
        ldn_individual_to_ethnicity[individual.get_id()] = 'white british';
    elif(individual.get_ethnicity_a()>1 and individual.get_ethnicity_a()<=5 and  individual.get_ethnicity_b()>1 and individual.get_ethnicity_b()<=18):
        ldn_ethnicity_minority = ldn_ethnicity_minority + 1;
        ldn_individual_to_ethnicity[individual.get_id()] = 'ethnic minority';

print(str(ldn_ethnicity_white_british) + ' ' + str(ldn_ethnicity_minority));

13962 29336


In [42]:
ldn_ethnicity_total = ldn_ethnicity_white_british + ldn_ethnicity_minority;
print(ldn_ethnicity_total);

43298


In [43]:
ldn_ethnicity_white_british_proportion = ldn_ethnicity_white_british / ldn_ethnicity_total;
ldn_ethnicity_minority_proportion = ldn_ethnicity_minority / ldn_ethnicity_total;
print(str(ldn_ethnicity_white_british_proportion) + ' ' + str(ldn_ethnicity_minority_proportion));

0.3224629313132246 0.6775370686867753


Distribution

In [44]:
ldn_ethnicity_white_british_proportion_100 = max(1, int(round(ldn_ethnicity_white_british_proportion, 2) * 100));
print("LDN white british: " + str(ldn_ethnicity_white_british_proportion_100));
ldn_ethnicity_minority_proportion_100 = max(1, int(round(ldn_ethnicity_minority_proportion, 2) * 100));
print("LDN minority ethnic: " + str(ldn_ethnicity_minority_proportion_100));

LDN white british: 32
LDN minority ethnic: 68


In [45]:
print(ldn_ethnicity_white_british_proportion_100 + ldn_ethnicity_minority_proportion_100);

100


<h3>Asthma</h3>

SEC

In [46]:
# function that attempts to gain consensus on SEC across the three metrics
def get_LDN_SEC(patient_id):
    classifications = [ldn_individual_to_lsoa_sec[patient_id] if patient_id in ldn_individual_to_lsoa_sec.keys() else "", ldn_individual_to_bmi_sec[patient_id] if patient_id in ldn_individual_to_bmi_sec.keys() else "", ldn_individual_to_covid_sec[patient_id] if patient_id in ldn_individual_to_covid_sec.keys() else ""];
    classifications_split = [item for sublist in list(map(lambda classification: classification.split(' or '), classifications)) for item in sublist];
    counts = {'low':classifications_split.count('low'), 'mid':classifications_split.count('mid'), 'high':classifications_split.count('high')};
    max_counts = [sec for sec, count in counts.items() if count == max(counts.values())];
    if(len(max_counts) == 1): return max_counts[0];
    elif(len(max_counts) == 2 and max_counts == ["low", "high"]): return "mid";
    return False;

In [47]:
asthma_patients = [];
other_patients = [];
with open(config.get('data', 'ldn.path_clusters'), encoding='utf-8') as f:
    reader = csv.DictReader(f);
    reader_by_date = sorted(reader, key=lambda row: datetime.strptime(row['effectivedate'], '%Y-%m-%d'));
    reader_by_date.reverse();
    for row in reader_by_date:
        if('AST' in row['cluster'] and not row['patient_id'] in asthma_patients):
            asthma_patients.append(row['patient_id']);
        elif(not 'AST' in row['cluster'] and not row['patient_id'] in asthma_patients and not row['patient_id'] in other_patients):
            other_patients.append(row['patient_id']);
# For interest
print(len(asthma_patients));
print(len(other_patients));

4342
22394


In [48]:
# Determine population of 6 groups of interest
ldn_sec_low_asthma_proportion, ldn_sec_mid_asthma_proportion, ldn_sec_high_asthma_proportion, ldn_sec_low_non_asthma_proportion, ldn_sec_mid_non_asthma_proportion, ldn_sec_high_non_asthma_proportion = 0, 0, 0, 0, 0, 0;
ldn_sec_low_asthma_individuals, ldn_sec_mid_asthma_individuals, ldn_sec_high_asthma_individuals, ldn_sec_low_non_asthma_individuals, ldn_sec_mid_non_asthma_individuals, ldn_sec_high_non_asthma_individuals = [], [], [], [], [], [];
for individual in individuals.values():
    sec = get_LDN_SEC(individual.get_id());
    if(sec):
        if(individual.get_id() in asthma_patients and sec=='low'):
            ldn_sec_low_asthma_proportion = ldn_sec_low_asthma_proportion + 1;
            ldn_sec_low_asthma_individuals.append(individual.get_id());
        elif(individual.get_id() in asthma_patients and sec=='mid'):
            ldn_sec_mid_asthma_proportion = ldn_sec_mid_asthma_proportion + 1;
            ldn_sec_mid_asthma_individuals.append(individual.get_id());
        elif(individual.get_id() in asthma_patients and sec=='high'):
            ldn_sec_high_asthma_proportion = ldn_sec_high_asthma_proportion + 1;
            ldn_sec_high_asthma_individuals.append(individual.get_id());
        elif(individual.get_id() in other_patients and sec=='low'):
            ldn_sec_low_non_asthma_proportion = ldn_sec_low_non_asthma_proportion + 1;
            ldn_sec_low_non_asthma_individuals.append(individual.get_id());
        elif(individual.get_id() in other_patients and sec=='mid'):
            ldn_sec_mid_non_asthma_proportion = ldn_sec_mid_non_asthma_proportion + 1;
            ldn_sec_mid_non_asthma_individuals.append(individual.get_id());
        elif(individual.get_id() in other_patients and sec=='high'):
            ldn_sec_high_non_asthma_proportion = ldn_sec_high_non_asthma_proportion + 1;
            ldn_sec_high_non_asthma_individuals.append(individual.get_id());

print(ldn_sec_low_asthma_proportion); 
print(ldn_sec_mid_asthma_proportion); 
print(ldn_sec_high_asthma_proportion); 
print(ldn_sec_low_non_asthma_proportion);
print(ldn_sec_mid_non_asthma_proportion); 
print(ldn_sec_high_non_asthma_proportion);
    

1031
1871
371
6175
9453
2011


In [49]:
ldn_sec_asthma_total = ldn_sec_low_asthma_proportion + ldn_sec_mid_asthma_proportion + ldn_sec_high_asthma_proportion + ldn_sec_low_non_asthma_proportion + ldn_sec_mid_non_asthma_proportion + ldn_sec_high_non_asthma_proportion;

Distributions

In [50]:
ldn_sec_low_asthma_proportion_100 = max(1, int(round(ldn_sec_low_asthma_proportion / ldn_sec_asthma_total, 2) * 100));
print("LDN low SEC + asthma: " + str(ldn_sec_low_asthma_proportion_100));
ldn_sec_mid_asthma_proportion_100 = max(1, int(round(ldn_sec_mid_asthma_proportion / ldn_sec_asthma_total, 2) * 100));
print("LDN mid SEC + asthma: " + str(ldn_sec_mid_asthma_proportion_100));
ldn_sec_high_asthma_proportion_100 = max(1, int(round(ldn_sec_high_asthma_proportion / ldn_sec_asthma_total, 2) * 100));
print("LDN high SEC + asthma: " + str(ldn_sec_high_asthma_proportion_100));
ldn_sec_low_non_asthma_proportion_100 = max(1, int(round(ldn_sec_low_non_asthma_proportion / ldn_sec_asthma_total, 2) * 100));
print("LDN low SEC: " + str(ldn_sec_low_non_asthma_proportion_100));
ldn_sec_mid_non_asthma_proportion_100 = max(1, int(round(ldn_sec_mid_non_asthma_proportion / ldn_sec_asthma_total, 2) * 100));
print("LDN mid SEC: " + str(ldn_sec_mid_non_asthma_proportion_100));
ldn_sec_high_non_asthma_proportion_100 = max(1, int(round(ldn_sec_high_non_asthma_proportion / ldn_sec_asthma_total, 2) * 100));
print("LDN high SEC: " + str(ldn_sec_high_non_asthma_proportion_100));

LDN low SEC + asthma: 5
LDN mid SEC + asthma: 9
LDN high SEC + asthma: 2
LDN low SEC: 30
LDN mid SEC: 45
LDN high SEC: 10


In [51]:
print(ldn_sec_low_asthma_proportion_100 + ldn_sec_mid_asthma_proportion_100 + ldn_sec_high_asthma_proportion_100 + ldn_sec_low_non_asthma_proportion_100 + ldn_sec_mid_non_asthma_proportion_100 + ldn_sec_high_non_asthma_proportion_100);

101


Ethnicity

In [52]:
# Determine population of 4 groups of interest
ldn_ethnicity_white_asthma_proportion, ldn_ethnicity_minority_asthma_proportion, ldn_ethnicity_white_non_asthma_proportion, ldn_ethnicity_minority_non_asthma_proportion = 0, 0, 0, 0;
ldn_ethnicity_white_asthma_individuals, ldn_ethnicity_minority_asthma_individuals, ldn_ethnicity_white_non_asthma_individuals, ldn_ethnicity_minority_non_asthma_individuals = [], [], [], [];
for individual in individuals.values():
    if(not individual.get_id() in ldn_individual_to_ethnicity.keys()): continue;
    ethnicity = ldn_individual_to_ethnicity[individual.get_id()];
    if(ethnicity):
        if(individual.get_id() in asthma_patients and ethnicity=='white british'):
            ldn_ethnicity_white_asthma_proportion = ldn_ethnicity_white_asthma_proportion + 1;
            ldn_ethnicity_white_asthma_individuals.append(individual.get_id());
        elif(individual.get_id() in asthma_patients and ethnicity=='ethnic minority'):
            ldn_ethnicity_minority_asthma_proportion = ldn_ethnicity_minority_asthma_proportion + 1;
            ldn_ethnicity_minority_asthma_individuals.append(individual.get_id());
        elif(individual.get_id() in other_patients and ethnicity=='white british'):
            ldn_ethnicity_white_non_asthma_proportion = ldn_ethnicity_white_non_asthma_proportion + 1;
            ldn_ethnicity_white_non_asthma_individuals.append(individual.get_id());
        elif(individual.get_id() in other_patients and ethnicity=='ethnic minority'):
            ldn_ethnicity_minority_non_asthma_proportion = ldn_ethnicity_minority_non_asthma_proportion + 1;
            ldn_ethnicity_minority_non_asthma_individuals.append(individual.get_id());

print(ldn_ethnicity_white_asthma_proportion); 
print(ldn_ethnicity_minority_asthma_proportion); 
print(ldn_ethnicity_white_non_asthma_proportion); 
print(ldn_ethnicity_minority_non_asthma_proportion);

824
2390
4163
10882


In [53]:
ldn_ethnicity_asthma_total = ldn_ethnicity_white_asthma_proportion + ldn_ethnicity_minority_asthma_proportion + ldn_ethnicity_white_non_asthma_proportion + ldn_ethnicity_minority_non_asthma_proportion

Distributions

In [54]:
ldn_ethnicity_minority_asthma_proportion_100 = max(1, int(round(ldn_ethnicity_minority_asthma_proportion / ldn_ethnicity_asthma_total, 2) * 100));
print("LDN minority ethnic + asthma: " + str(ldn_sec_mid_asthma_proportion_100));
ldn_ethnicity_white_asthma_proportion_100 = max(1, int(round(ldn_ethnicity_white_asthma_proportion / ldn_ethnicity_asthma_total, 2) * 100));
print("LDN white british + asthma: " + str(ldn_sec_low_asthma_proportion_100));
ldn_ethnicity_minority_non_asthma_proportion_100 = max(1, int(round(ldn_ethnicity_minority_non_asthma_proportion / ldn_ethnicity_asthma_total, 2) * 100));
print("LDN minority ethnic: " + str(ldn_ethnicity_minority_non_asthma_proportion_100));
ldn_ethnicity_white_non_asthma_proportion_100 = max(1, int(round(ldn_ethnicity_white_non_asthma_proportion / ldn_ethnicity_asthma_total, 2) * 100));
print("LDN white british: " + str(ldn_sec_low_non_asthma_proportion_100));

LDN minority ethnic + asthma: 9
LDN white british + asthma: 5
LDN minority ethnic: 60
LDN white british: 30


In [55]:
print(ldn_ethnicity_white_asthma_proportion_100 + ldn_ethnicity_minority_asthma_proportion_100 + ldn_ethnicity_white_non_asthma_proportion_100 + ldn_ethnicity_minority_non_asthma_proportion_100);

101


<h3>Health utilisation</h3>

In [56]:
def is_winter(month):
    if(month==12 or month==1 or month==2): return True;
    return False;

def get_total_utilisation(row):
    utilisation_keys = ['gp_f2f', 'gp_tel', 'gp_hv', 'gp_ett', 'nurse_f2f', 'nurse_tel', 'nurse_hv', 'nurse_ett', 'ohcpuc_f2f', 'ohcpuc_tel', 'ohcpuc_hv', 'ohcpuc_ett'];
    return sum(dict(map(lambda cell:(cell[0],int(cell[1])), filter(lambda cell:cell[0] in utilisation_keys, row.items()))).values());

In [57]:
# for interest
# compare utilisation in winter months with rest of year
year_to_utilisation_winter = {};
year_to_utilisation_other = {};
with open(config.get('data', 'ldn.path_consultations'), encoding='utf-8') as f:
    reader = csv.DictReader(f);
    for row in reader:
        total_utilisation = get_total_utilisation(row);
        if(is_winter(int(row['con_month']))):
            year_to_utilisation_winter[row['con_year']] = year_to_utilisation_winter.setdefault(row['con_year'], 0) + total_utilisation;
        else:
            year_to_utilisation_other[row['con_year']] = year_to_utilisation_other.setdefault(row['con_year'], 0) + total_utilisation;

print(str(year_to_utilisation_winter) + ' ' + str(year_to_utilisation_other));

{'2018': 41612, '2017': 12426, '2019': 47488, '2020': 49244, '2021': 57936, '2022': 29718} {'2017': 99066, '2018': 118186, '2019': 131553, '2020': 140171, '2021': 205833}


In [58]:
print(sum(year_to_utilisation_winter.values())/len(year_to_utilisation_winter.values()));
print(sum(year_to_utilisation_other.values())/len(year_to_utilisation_other.values()));

39737.333333333336
138961.8


In [59]:
def record_utilisation_sec_asthma(patient_id, year, ldn_utilisation, total_utilisation):
    ldn_utilisation.setdefault(year, {});
    if(patient_id in ldn_sec_low_asthma_individuals):
        ldn_utilisation[row['con_year']].setdefault('sec_low_asthma', []).append(total_utilisation);
    elif(patient_id in ldn_sec_mid_asthma_individuals):
        ldn_utilisation[row['con_year']].setdefault('sec_mid_asthma', []).append(total_utilisation);
    elif(patient_id in ldn_sec_high_asthma_individuals):
        ldn_utilisation[row['con_year']].setdefault('sec_high_asthma', []).append(total_utilisation);
    elif(patient_id in ldn_sec_low_non_asthma_individuals):
        ldn_utilisation[row['con_year']].setdefault('sec_low_non_asthma', []).append(total_utilisation);
    elif(patient_id in ldn_sec_mid_non_asthma_individuals):
        ldn_utilisation[row['con_year']].setdefault('sec_mid_non_asthma', []).append(total_utilisation);
    elif(patient_id in ldn_sec_high_non_asthma_individuals):
        ldn_utilisation[row['con_year']].setdefault('sec_high_non_asthma', []).append(total_utilisation);

def record_utilisation_ethnicity_asthma(patient_id, year, ldn_utilisation, total_utilisation):
    ldn_utilisation.setdefault(year, {});
    if(patient_id in ldn_ethnicity_white_asthma_individuals):
        ldn_utilisation[row['con_year']].setdefault('ethnicity_white_asthma', []).append(total_utilisation);
    elif(patient_id in ldn_ethnicity_minority_asthma_individuals):
        ldn_utilisation[row['con_year']].setdefault('ethnicity_minority_asthma', []).append(total_utilisation);
    elif(patient_id in ldn_ethnicity_white_non_asthma_individuals):
        ldn_utilisation[row['con_year']].setdefault('ethnicity_white_non_asthma', []).append(total_utilisation);
    elif(patient_id in ldn_ethnicity_minority_non_asthma_individuals):
        ldn_utilisation[row['con_year']].setdefault('ethnicity_minority_non_asthma', []).append(total_utilisation);

In [60]:
# break down utilisation by each group above
ldn_utilisation_winter = {};
ldn_utilisation_other = {};

with open(config.get('data', 'ldn.path_consultations'), encoding='utf-8') as f:
    reader = csv.DictReader(f);
    for row in reader:
        total_utilisation = get_total_utilisation(row);
        patient_id = row['patient_id'];
        if(is_winter(int(row['con_month']))): 
            record_utilisation_sec_asthma(patient_id, row['con_year'], ldn_utilisation_winter, total_utilisation);
            record_utilisation_ethnicity_asthma(patient_id, row['con_year'], ldn_utilisation_winter, total_utilisation);
        else: 
            record_utilisation_sec_asthma(patient_id, row['con_year'], ldn_utilisation_other, total_utilisation);
            record_utilisation_ethnicity_asthma(patient_id, row['con_year'], ldn_utilisation_other, total_utilisation);

In [61]:
# Get average visits per person per year within each group
ldn_utilisation_winter_average = dict(map(lambda year_entries:(year_entries[0], dict(map(lambda group_entries:(group_entries[0], sum(group_entries[1])/len(group_entries[1])), year_entries[1].items()))), ldn_utilisation_winter.items()));
ldn_utilisation_other_average = dict(map(lambda year_entries:(year_entries[0], dict(map(lambda group_entries:(group_entries[0], sum(group_entries[1])/len(group_entries[1])), year_entries[1].items()))), ldn_utilisation_other.items()));
print(ldn_utilisation_winter_average);
print(ldn_utilisation_other_average);

{'2018': {'ethnicity_white_non_asthma': 1.55718879191548, 'sec_mid_non_asthma': 1.5250719276613234, 'ethnicity_minority_non_asthma': 1.489612676056338, 'sec_high_asthma': 1.5304347826086957, 'ethnicity_minority_asthma': 1.5448504983388704, 'sec_low_non_asthma': 1.4891304347826086, 'sec_mid_asthma': 1.5613040396881643, 'sec_low_asthma': 1.6243654822335025, 'ethnicity_white_asthma': 1.5884244372990353, 'sec_high_non_asthma': 1.5828402366863905}, '2017': {'sec_mid_non_asthma': 1.5130434782608695, 'ethnicity_minority_non_asthma': 1.490096208262592, 'sec_high_asthma': 1.295774647887324, 'ethnicity_minority_asthma': 1.5156794425087108, 'sec_mid_asthma': 1.5601750547045952, 'sec_low_non_asthma': 1.4771739130434782, 'sec_low_asthma': 1.4067796610169492, 'sec_high_non_asthma': 1.5521472392638036, 'ethnicity_white_non_asthma': 1.5093795093795095, 'ethnicity_white_asthma': 1.4564102564102563}, '2019': {'sec_mid_non_asthma': 1.561774616248596, 'ethnicity_minority_non_asthma': 1.5178600554558799, '

Utilisation

In [62]:
# Winter, SEC + asthma
# Average utilisation across the years
sec_low_asthma_utilisation_winter = sum(list(map(lambda year:year['sec_low_asthma'], ldn_utilisation_winter_average.values()))) / len(ldn_utilisation_winter_average.values());
print(sec_low_asthma_utilisation_winter);

sec_mid_asthma_utilisation_winter = sum(list(map(lambda year:year['sec_mid_asthma'], ldn_utilisation_winter_average.values()))) / len(ldn_utilisation_winter_average.values());
print(sec_mid_asthma_utilisation_winter);

sec_high_asthma_utilisation_winter = sum(list(map(lambda year:year['sec_high_asthma'], ldn_utilisation_winter_average.values()))) / len(ldn_utilisation_winter_average.values());
print(sec_high_asthma_utilisation_winter);

sec_low_non_asthma_utilisation_winter = sum(list(map(lambda year:year['sec_low_non_asthma'], ldn_utilisation_winter_average.values()))) / len(ldn_utilisation_winter_average.values());
print(sec_low_non_asthma_utilisation_winter);

sec_mid_non_asthma_utilisation_winter = sum(list(map(lambda year:year['sec_mid_non_asthma'], ldn_utilisation_winter_average.values()))) / len(ldn_utilisation_winter_average.values());
print(sec_mid_non_asthma_utilisation_winter);

sec_high_non_asthma_utilisation_winter = sum(list(map(lambda year:year['sec_high_non_asthma'], ldn_utilisation_winter_average.values()))) / len(ldn_utilisation_winter_average.values());
print(sec_high_non_asthma_utilisation_winter);


1.7013797308156995
1.6639995994585057
1.6648814625571289
1.641581665242377
1.6746297557045875
1.6842473517357963


In [63]:

# Winter, ethnicity + asthma
print(sum(list(map(lambda year:year['ethnicity_white_asthma'], ldn_utilisation_winter_average.values()))) / len(ldn_utilisation_winter_average.values()));
print(sum(list(map(lambda year:year['ethnicity_minority_asthma'], ldn_utilisation_winter_average.values()))) / len(ldn_utilisation_winter_average.values()));
print(sum(list(map(lambda year:year['ethnicity_white_non_asthma'], ldn_utilisation_winter_average.values()))) / len(ldn_utilisation_winter_average.values()));
print(sum(list(map(lambda year:year['ethnicity_minority_non_asthma'], ldn_utilisation_winter_average.values()))) / len(ldn_utilisation_winter_average.values()));

1.7514018506184623
1.6562808425103388
1.7128859121354119
1.6259644858115239


In [64]:
# Other months, SEC + asthma
print(sum(list(map(lambda year:year['sec_low_asthma'], ldn_utilisation_other_average.values()))) / len(ldn_utilisation_other_average.values()));
print(sum(list(map(lambda year:year['sec_mid_asthma'], ldn_utilisation_other_average.values()))) / len(ldn_utilisation_other_average.values()));
print(sum(list(map(lambda year:year['sec_high_asthma'], ldn_utilisation_other_average.values()))) / len(ldn_utilisation_other_average.values()));
print(sum(list(map(lambda year:year['sec_low_non_asthma'], ldn_utilisation_other_average.values()))) / len(ldn_utilisation_other_average.values()));
print(sum(list(map(lambda year:year['sec_mid_non_asthma'], ldn_utilisation_other_average.values()))) / len(ldn_utilisation_other_average.values()));
print(sum(list(map(lambda year:year['sec_high_non_asthma'], ldn_utilisation_other_average.values()))) / len(ldn_utilisation_other_average.values()));

1.6579320720780832
1.616808385162713
1.6592940022009972
1.6404330447965563
1.6796514857816451
1.6941100170822039


In [65]:
# Other months, ethnicity + asthma
print(sum(list(map(lambda year:year['ethnicity_minority_asthma'], ldn_utilisation_other_average.values()))) / len(ldn_utilisation_other_average.values()));
print(sum(list(map(lambda year:year['ethnicity_white_asthma'], ldn_utilisation_other_average.values()))) / len(ldn_utilisation_other_average.values()));
print(sum(list(map(lambda year:year['ethnicity_minority_non_asthma'], ldn_utilisation_other_average.values()))) / len(ldn_utilisation_other_average.values()));
print(sum(list(map(lambda year:year['ethnicity_white_non_asthma'], ldn_utilisation_other_average.values()))) / len(ldn_utilisation_other_average.values()));

1.6261448748506369
1.6715162901091305
1.6461836148265057
1.6938452322296236
